# ConversationBufferWindowMemory

`ConversationBufferWindowMemory` 保留了对话过程中交互的列表。它仅使用最后 K 次交互。这对于保持最近交互的滑动窗口非常有用，因此缓冲区不会变得太大。

让我们首先探索这种记忆类型的基本功能。

In [3]:
from langchain.memory import ConversationBufferWindowMemory

In [5]:
memory = ConversationBufferWindowMemory( k=1)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

In [6]:
memory.load_memory_variables({})

{'history': 'Human: not much you\nAI: not much'}

我们还可以将历史记录作为消息列表获取（如果您正在使用聊天模型，则这很有用）。

In [8]:
memory = ConversationBufferWindowMemory( k=1, return_messages=True)
memory.save_context({"input": "hi"}, {"output": "whats up"})
memory.save_context({"input": "not much you"}, {"output": "not much"})

In [9]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='not much you', additional_kwargs={}),
  AIMessage(content='not much', additional_kwargs={})]}

## 使用链式结构
让我们通过一个示例来了解，同样设置 `verbose=True`，以便我们可以看到提示。

In [12]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
conversation_with_summary = ConversationChain(
    llm=OpenAI(temperature=0), 
    # We set a low k=2, to only keep the last 2 interactions in memory
    memory=ConversationBufferWindowMemory(k=2), 
    verbose=True
)
conversation_with_summary.predict(input="Hi, what's up?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, what's up?
AI:

> Finished chain.


" Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?"

In [13]:
conversation_with_summary.predict(input="What's their issues?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up?
AI:  Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?
Human: What's their issues?
AI:

> Finished chain.


" The customer is having trouble connecting to their Wi-Fi network. I'm helping them troubleshoot the issue and get them connected."

In [14]:
conversation_with_summary.predict(input="Is it going well?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, what's up?
AI:  Hi there! I'm doing great. I'm currently helping a customer with a technical issue. How about you?
Human: What's their issues?
AI:  The customer is having trouble connecting to their Wi-Fi network. I'm helping them troubleshoot the issue and get them connected.
Human: Is it going well?
AI:

> Finished chain.


" Yes, it's going well so far. We've already identified the problem and are now working on a solution."

In [15]:
# Notice here that the first interaction does not appear.
conversation_with_summary.predict(input="What's the solution?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What's their issues?
AI:  The customer is having trouble connecting to their Wi-Fi network. I'm helping them troubleshoot the issue and get them connected.
Human: Is it going well?
AI:  Yes, it's going well so far. We've already identified the problem and are now working on a solution.
Human: What's the solution?
AI:

> Finished chain.


" The solution is to reset the router and reconfigure the settings. We're currently in the process of doing that."